In [1]:

from itertools import permutations
from joblib import Parallel, delayed
from more_itertools import distinct_permutations
from tqdm import tqdm
import numpy as np
import pickle

from qulacs import QuantumCircuit

from mcr.equiv_check import (
    equiv,
    pauli_bit_equivalence_check,
    equivalence_check_via_mqt_qcec,
)

from mcr.gate_apply import (
    PauliBit,
    grouping,
    loop_optimization,
    set_clifford_to_qulacs,
    zhang_optimization,
)

from opt_using_mcr import test_algorithm
from perform_mcr import (
    attempt_mcr_retry,
    three_layer_nontrivial_swap,
)
from optimizer import (
    full_optimization,
    optimize_data_loop,
    mcr_swap,
    have_common_pauli_str,
)

In [2]:
filetype = "seq"  # "small" or "seq"
nqubits = 20  # Number of qubits in the circuit
# with open(f"unopt_{filetype}.pickle", "rb") as f:
with open(f"unopt_{nqubits}.pickle", "rb") as f:
    seq = pickle.load(f)
data = []
for elem in seq:
    sgn = str(elem[1])[0]
    pauli_str = str(elem[1])[1:]
    if sgn == "+":
        data.append(PauliBit(pauli_str, np.pi/4))
    else:
        assert sgn == "-", f"Unexpected sign: {sgn}"
        data.append(PauliBit(pauli_str, -np.pi/4))
data.append(PauliBit("Z" * nqubits, -np.pi / 4))  # Add identity gate

In [3]:
def load_data_from_pickle(filepath: str, nqubits: int) -> list:
    """pickle形式のファイルを読み込む

    Args:
        filepath (str): pickle形式のファイルパス

    Returns:
        list: 読み込んだデータのリスト
    """
    with open(filepath, mode="br") as f:
        data = pickle.load(f)
    pauli_bit_sequences = []
    for elem in data:
        initial_pauli_str, theta, qubit_indices = elem
        pauli_str = ["I"] * nqubits
        for pauli, qubit_idx in zip(initial_pauli_str, qubit_indices):
            pauli_str[qubit_idx] = pauli
        pauli_bit = PauliBit("".join(pauli_str), theta)
        pauli_bit_sequences.append(pauli_bit)
    return pauli_bit_sequences
nqubits = 6
# data = load_data_from_pickle("te_pai_data.pickle", nqubits=nqubits)

In [4]:
clifford, non_clifford = full_optimization(data, max_iter=3, show_opt_log=True)

🔁 Optimization iteration: 1 / 3
🎉 Optimization success in iteration 1: 4796 → 3198
🎉 Optimization success in iteration 2: 3198 → 2008
🎉 Optimization success in iteration 3: 2008 → 1178
🎉 Optimization success in iteration 4: 1178 → 806
🎉 Optimization success in iteration 5: 806 → 586
🎉 Optimization success in iteration 6: 586 → 458
🎉 Optimization success in iteration 7: 458 → 350
🎉 Optimization success in iteration 8: 350 → 280
🎉 Optimization success in iteration 9: 280 → 220
🎉 Optimization success in iteration 10: 220 → 182
🎉 Optimization success in iteration 11: 182 → 166
🎉 Optimization success in iteration 12: 166 → 148
🎉 Optimization success in iteration 13: 148 → 132
🎉 Optimization success in iteration 14: 132 → 130
🔍 No optimization in iteration 15: 130 → 130
⚙️  Additional optimization: 1 / 3
🎉 Optimization success in iteration 1: 154 → 124
🎉 Optimization success in iteration 2: 124 → 108
🎉 Optimization success in iteration 3: 108 → 100
🎉 Optimization success in iteration 4: 100 

In [5]:
non_clifford

[ZXZYXXIZIZXYXXYXXZII, 0.785,
 YZIIIYYXZYYIYIXXYZXY, -0.785,
 YZYXYZZIIYXYIZXZYZZX, -0.785,
 YZIZXIIIYXYZXYIIZXIZ, 0.785,
 ZIYYIXYYXZZXZXXIIZZY, 0.785,
 ZIYYIZZYYYYYIIYIYYXZ, 0.785,
 YXYIIXIIZYZZYXIXIZIY, -0.785,
 IYIXYYZIZIYXYYXYYIZZ, 0.785,
 XXXYIYZIXIYZIYIYIXXX, 0.785,
 YXYIIXIIZYZZYXIXIZIY, 0.785,
 ZIXIZIIIIXYXZYXIXIYX, -0.785,
 IIYYZZZIYZZXXXXZXYZY, -0.785,
 IYIXYYZIZIYXYYXYYIZZ, -0.785,
 YZYXYZZIIYXYIZXZYZZX, 0.785,
 ZIZYIZZIYYXIYZIZIYXZ, 0.785,
 IYIXYYZIZIYXYYXYYIZZ, -0.785,
 YXIXYIXXIYIXIYIZIZYX, 0.785,
 YXIYZIYIYIXXIXYYZIXX, -0.785,
 IYZZXXIYXZYYXIYYXYIX, 0.785,
 ZIIIZYXYZIIIYIIZXXII, -0.785,
 YXIYZIYIYIXXIXYYZIXX, 0.785,
 ZZZZZZZZZZZZZZZZZZZZ, -0.785]

In [6]:
m = grouping(non_clifford)
m

[[ZXZYXXIZIZXYXXYXXZII, 0.785],
 [YZIIIYYXZYYIYIXXYZXY, -0.785, YZYXYZZIIYXYIZXZYZZX, -0.785],
 [YZIZXIIIYXYZXYIIZXIZ, 0.785,
  ZIYYIXYYXZZXZXXIIZZY, 0.785,
  ZIYYIZZYYYYYIIYIYYXZ, 0.785],
 [YXYIIXIIZYZZYXIXIZIY, -0.785],
 [IYIXYYZIZIYXYYXYYIZZ, 0.785, XXXYIYZIXIYZIYIYIXXX, 0.785],
 [YXYIIXIIZYZZYXIXIZIY, 0.785, ZIXIZIIIIXYXZYXIXIYX, -0.785],
 [IIYYZZZIYZZXXXXZXYZY, -0.785, IYIXYYZIZIYXYYXYYIZZ, -0.785],
 [YZYXYZZIIYXYIZXZYZZX, 0.785, ZIZYIZZIYYXIYZIZIYXZ, 0.785],
 [IYIXYYZIZIYXYYXYYIZZ, -0.785],
 [YXIXYIXXIYIXIYIZIZYX, 0.785, YXIYZIYIYIXXIXYYZIXX, -0.785],
 [IYZZXXIYXZYYXIYYXYIX, 0.785, ZIIIZYXYZIIIYIIZXXII, -0.785],
 [YXIYZIYIYIXXIXYYZIXX, 0.785, ZZZZZZZZZZZZZZZZZZZZ, -0.785]]

In [10]:
# equiv([[],data],[clifford, non_clifford])

In [ ]:
# itertools.permutations で全列挙は計算量が膨大なので、numpy配列でインデックスの順列を生成し、必要な部分だけ計算することで高速化します。
seq_a = [
    PauliBit("ix", -np.pi / 4),
    PauliBit("zi", -np.pi / 4),
    PauliBit("xy", np.pi / 4),
    PauliBit("zy", -np.pi / 4),
    PauliBit("yy", -np.pi / 4),
    PauliBit("zz", -np.pi / 4),
    PauliBit("ix", np.pi / 4),
    PauliBit("xi", np.pi / 4),
    PauliBit("yz", -np.pi / 4),
    PauliBit("xz", -np.pi / 4),
    PauliBit("zz", -np.pi / 4),
]
seq_b = [PauliBit("yx", -np.pi / 4)]
equiv([[], seq_a], [[],seq_b])

False

In [ ]:
A = [
    PauliBit("IZ", -np.pi / 4),
    PauliBit("ZY", np.pi / 4),
    PauliBit("XY", -np.pi / 4),
    PauliBit("IZ", -np.pi / 4),
    PauliBit("XZ", -np.pi / 4),
    PauliBit("IY", -np.pi / 4),
    PauliBit("YZ", np.pi / 4),
    PauliBit("IX", -np.pi / 4),
    PauliBit("ZI", np.pi / 4),
    PauliBit("YY", -np.pi / 4),
    PauliBit("XI", -np.pi / 4),
    PauliBit("ZI", -np.pi / 4),
    PauliBit("ZZ", np.pi / 4),
    PauliBit("XI", np.pi / 4),
    PauliBit("ZX", np.pi / 4),
    PauliBit("XX", np.pi / 4),
]

In [ ]:
m = grouping(A)
# m[8] += [PauliBit("XZ", -np.pi / 4)]
# m.insert(8, [PauliBit("XZ", np.pi / 4)])
m[8] += [PauliBit("XZ", np.pi / 4), PauliBit("XZ", -np.pi / 4)]
# m.insert(8, [PauliBit("XZ", -np.pi / 4)])
m

[[IZ, -0.785],
 [ZY, 0.785],
 [XY, -0.785],
 [IZ, -0.785, XZ, -0.785],
 [IY, -0.785],
 [YZ, 0.785],
 [IX, -0.785, ZI, 0.785],
 [YY, -0.785],
 [XI, -0.785, XZ, 0.785, XZ, -0.785],
 [ZI, -0.785, ZZ, 0.785],
 [XI, 0.785],
 [ZX, 0.785],
 [XX, 0.785]]

In [ ]:
# ini = m.copy()
seq = mcr_swap(m)
clifford_lst, optimized_data = optimize_data_loop(
    seq, show_opt_log=True, max_attempts=3
)


update_MCR
left_bits: (XI, -0.785, XZ, 0.785), right_bits: (ZI, -0.785, ZZ, 0.785)
only swap
left_bits: (IX, -0.785, ZI, 0.785), right_bits: (YY, -0.785, XZ, -0.785)
only swap
left_bits: (ZI, -0.785, ZZ, 0.785), right_bits: (XI, 0.785, XZ, 0.785)
🎉 Optimization success in iteration 1: 19 → 18
only swap
left_bits: (XZ, -0.785, YY, -0.785), right_bits: (IX, -0.785, ZI, 0.785)
🎉 Optimization success in iteration 2: 18 → 16
🔍 No optimization in iteration 3: 16 → 16
🔍 No optimization in iteration 3: 16 → 16
🔍 No optimization in iteration 3: 16 → 16


In [ ]:
clifford_lst, optimized_data = optimize_data_loop(
    seq, show_opt_log=True, max_attempts=3
)


only swap
left_bits: (IX, -0.785, ZI, 0.785), right_bits: (YY, -0.785, XZ, -0.785)
only swap
left_bits: (ZI, -0.785, ZZ, 0.785), right_bits: (XI, 0.785, XZ, 0.785)
🎉 Optimization success in iteration 1: 19 → 18
only swap
left_bits: (XZ, -0.785, YY, -0.785), right_bits: (IX, -0.785, ZI, 0.785)
🎉 Optimization success in iteration 2: 18 → 16
🔍 No optimization in iteration 3: 16 → 16
🔍 No optimization in iteration 3: 16 → 16
🔍 No optimization in iteration 3: 16 → 16


In [ ]:
optimized_data

[IZ, -0.785,
 ZY, 0.785,
 XY, -0.785,
 IZ, -0.785,
 XZ, -0.785,
 IY, -0.785,
 YZ, 0.785,
 IX, -0.785,
 ZI, 0.785,
 YY, -0.785,
 XI, -0.785,
 ZI, -0.785,
 ZZ, 0.785,
 XI, 0.785,
 ZX, 0.785,
 XX, 0.785]

In [ ]:
three_layer_nontrivial_swap(grouping(A),with_mcr_index=True)

([IZ, -0.785,
  ZY, 0.785,
  IY, -0.785,
  IZ, -0.785,
  XZ, -0.785,
  XY, -0.785,
  YZ, 0.785,
  IX, -0.785,
  ZI, 0.785,
  YY, -0.785,
  XI, -0.785,
  ZI, -0.785,
  ZZ, 0.785,
  XI, 0.785,
  ZX, 0.785,
  XX, 0.785],
 {2})

In [ ]:
p =grouping(optimize_data_loop(
    three_layer_nontrivial_swap(grouping(A)), show_opt_log=True
)[1])
p

only swap
left_bits: (XY, -0.785, YZ, 0.785), right_bits: (IX, -0.785, ZI, 0.785)
🔍 No optimization in iteration 1: 16 → 16


[[IZ, -0.785],
 [IY, -0.785, ZY, 0.785],
 [IZ, -0.785, XZ, -0.785],
 [IX, -0.785, ZI, 0.785],
 [XY, -0.785, YZ, 0.785],
 [YY, -0.785],
 [XI, -0.785],
 [ZI, -0.785, ZZ, 0.785],
 [XI, 0.785],
 [ZX, 0.785],
 [XX, 0.785]]

In [ ]:
seq_a = [
    PauliBit("ix", -np.pi / 4),
    PauliBit("xy", np.pi / 4),
    PauliBit("zy", -np.pi / 4),
    PauliBit("yy", -np.pi / 4),
    PauliBit("zz", -np.pi / 4),
    PauliBit("ix", np.pi / 4),
    PauliBit("xi", np.pi / 4),
    PauliBit("yz", -np.pi / 4),
    PauliBit("xz", -np.pi / 4),
    PauliBit("zz", -np.pi / 4),
]

# seq_b = [
#     PauliBit("XZ", np.pi / 4),
#     PauliBit("ZI", -np.pi / 4),
#     PauliBit("ZZ", np.pi / 4),
#     PauliBit("XZ", -np.pi / 4),
# ]
# equiv([[], seq_a], [[], seq_b])

In [ ]:
m = grouping(seq_a)
m

[[IX, -0.785],
 [XY, 0.785],
 [ZY, -0.785],
 [YY, -0.785, ZZ, -0.785],
 [IX, 0.785, XI, 0.785],
 [YZ, -0.785],
 [XZ, -0.785],
 [ZZ, -0.785]]

In [ ]:

m[2] += [PauliBit("YZ", np.pi / 4), PauliBit("YZ", -np.pi / 4)]
# optimize_data_loop(mcr_swap(m))
mcr_swap(m)

update_MCR
left_bits: (ZY, -0.785, YZ, 0.785), right_bits: (YY, -0.785, ZZ, -0.785)
update_MCR
left_bits: (ZY, 0.785, YZ, 0.785), right_bits: (IX, 0.785, XI, 0.785)


[IX, -0.785,
 XY, 0.785,
 YZ, -0.785,
 ZY, -1.571,
 YY, -0.785,
 ZZ, -0.785,
 ZY, 1.571,
 IX, 0.785,
 XI, 0.785,
 ZY, -0.785,
 YZ, 0.785,
 YZ, -0.785,
 XZ, -0.785,
 ZZ, -0.785]